<a href="https://colab.research.google.com/github/DanaJian/Machine-learning-technologies/blob/main/Data_preprocessing_Feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [126]:
#Загрузка данных
data = pd.read_csv('/content/train.csv')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [127]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

1. Реализация функций OneHotEncoding и Softmax

In [128]:
def OneHotEncoding(df: pd.DataFrame, column_names: List[str]) -> pd.DataFrame:
  for column_name in column_names:
    column = df[column_name]
    unique_values = column.unique()
    n_values = len(unique_values)
    one_hot_encoded = np.zeros((len(column), n_values))
    
    for i, value in enumerate(unique_values):
        one_hot_encoded[:, i] = column == value
    
    one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=[f"{column_name}_{value}" for value in unique_values])
    
    df = pd.concat([df, one_hot_encoded_df], axis=1).drop(column_name, axis=1)
    df.drop_duplicates()
  return df


def Softmax(data):
  return np.exp(data) / np.sum(np.exp(data), axis=1, keepdims=True)

2. ONE-HOT-ENCODING

In [129]:
# Создание столбца признаков социального положения человека из столбца Name
data['Social_Status'] = data['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

data = OneHotEncoding(
                  df = data, 
                  column_names = ['Sex', 'Embarked', 'Social_Status']
               )

data.head(3)

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Social_Status_Mme,Social_Status_Ms,Social_Status_Major,Social_Status_Lady,Social_Status_Sir,Social_Status_Mlle,Social_Status_Col,Social_Status_Capt,Social_Status_the Countess,Social_Status_Jonkheer
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


3. DATA PREPROCESSING

In [130]:
nan_columns = data.columns[data.isnull().any()].tolist()
print('Столбцы со значением NaN:', nan_columns)

# Заполнение пропущенных значений в столбце Age медианным значением
median_age = data['Age'].median()
data.fillna({'Age': median_age}, inplace=True)

Столбцы со значением NaN: ['Age', 'Cabin']


In [131]:
# Удаление столбцов Name, PassengerId, Ticket и Cabin с чрезмерной информацией
cols_to_drop = ['Name', 'PassengerId', 'Ticket', 'Cabin']
for col in cols_to_drop:
    if col in data.columns:
        data = data.drop(col, axis=1)

In [132]:
# Сохранение копии данных
raw_data = data.copy()

In [133]:
# Масштабирование данных в столбцах Age и Fare
scaler = StandardScaler()
data[['Age', 'Fare']] = scaler.fit_transform(data[['Age', 'Fare']])

#Нормализация данных
normalizer = MinMaxScaler()
data[['Age', 'Fare']] = normalizer.fit_transform(data[['Age', 'Fare']])

data

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Sex_female,Embarked_S,Embarked_C,...,Social_Status_Mme,Social_Status_Ms,Social_Status_Major,Social_Status_Lady,Social_Status_Sir,Social_Status_Mlle,Social_Status_Col,Social_Status_Capt,Social_Status_the Countess,Social_Status_Jonkheer
0,0,3,0.271174,1,0,0.014151,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.472229,1,0,0.139136,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,0.321438,0,0,0.015469,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,0.434531,1,0,0.103644,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,3,0.434531,0,0,0.015713,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0.334004,0,0,0.025374,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
887,1,1,0.233476,0,0,0.058556,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0,3,0.346569,1,2,0.045771,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1,1,0.321438,0,0,0.058556,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


4. Оценка качества на предобработанных данных

In [134]:
for test_size in range(100):
  X_train, X_test, y_train, y_test = train_test_split(data.drop('Survived', axis=1), data['Survived'], test_size=0.2)

  lr = LogisticRegression(random_state=42)
  lr.fit(X_train, y_train)
  lr = lr.score(X_test, y_test)

print('Accuracy:', lr)

Accuracy: 0.8156424581005587


5. Оценка качества на непредобработанных данных

In [135]:
for test_size in range(100):
  X_train, X_test, y_train, y_test = train_test_split(raw_data.drop('Survived', axis=1), raw_data['Survived'], test_size=0.2)

  raw_lr = LogisticRegression()
  raw_lr.fit(X_train, y_train)
  rl = raw_lr.score(X_test, y_test)

print('Accuracy raw data:', rl)

Accuracy raw data: 0.7877094972067039


6. Оценка качества на числовых данных

In [136]:
for test_size in range(100):
  X_train, X_test, y_train, y_test = train_test_split(data[['Age', 'Fare']], data['Survived'], test_size=0.2)

  numeric_lr = LogisticRegression()
  numeric_lr.fit(X_train, y_train)
  nl = numeric_lr.score(X_test, y_test)

print('Accuracy numeric data:', nl)

Accuracy numeric data: 0.5865921787709497


7. ВЫВОД

In [137]:
data = {
    'Предобработанные': lr,
    'Непредобработанные': rl,
    'Числовые': nl,
    'x': np.arange(100),
}

results_df = pd.DataFrame(data)
results_df

,Предобработанные,Непредобработанные,Числовые,x
0,0.815642,0.787709,0.586592,0
1,0.815642,0.787709,0.586592,1
2,0.815642,0.787709,0.586592,2
3,0.815642,0.787709,0.586592,3
4,0.815642,0.787709,0.586592,4
...,...,...,...,...
95,0.815642,0.787709,0.586592,95
96,0.815642,0.787709,0.586592,96
97,0.815642,0.787709,0.586592,97
98,0.815642,0.787709,0.586592,98
